In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [2]:
# Define LeNet-5 architecture

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.act2 = nn.Tanh()
        self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1)
        self.act3 = nn.Tanh()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.avgpool1(self.act1(self.conv1(x)))
        x = self.avgpool2(self.act2(self.conv2(x)))
        x = self.flatten(self.act3(self.conv3(x)))
        x = self.act4(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

100%|██████████| 9912422/9912422 [00:00<00:00, 78034395.07it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 4814231.53it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 6186297.97it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5439899.71it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# Initialize LeNet-5 model

lenet5_model = LeNet5()

In [6]:
# Define loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet5_model.parameters(), lr=0.01, momentum=0.9)

In [9]:
# Training the model

for epoch in range(5):  # Adjust the number of epochs as needed
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), desc=f"epoch - {epoch+1}"):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = lenet5_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f" Loss: {running_loss / len(trainloader)}")

print("Finished Training")

epoch - 1: 938it [00:36, 25.65it/s]

 Loss: 0.31958595392451106



epoch - 2: 938it [00:33, 27.71it/s]

 Loss: 0.10398434807778771



epoch - 3: 938it [00:33, 28.41it/s]

 Loss: 0.07171648211612391



epoch - 4: 938it [00:31, 29.34it/s]

 Loss: 0.05567996582087836



epoch - 5: 938it [00:33, 28.21it/s]

 Loss: 0.04744717180192756
Finished Training


In [10]:
# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = lenet5_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total}%")

Accuracy on the test set: 98.33%


That's all. thankss !